In [0]:
%run ./customlogs

In [0]:
import pyspark
from pyspark.sql import SparkSession


In [0]:
dbutils.fs.unmount("/mnt/dbdemo")

/mnt/dbdemo has been unmounted.
Out[19]: True

In [0]:
container_name = <container_name>
storage_account_name = <account_name>
sas = <access_key>
url = "wasbs://" + container_name + "@" + storage_account_name + ".blob.core.windows.net"
config = "fs.azure.sas." + container_name + "." + storage_account_name + ".blob.core.windows.net"

In [0]:
  dbutils.fs.mount(  
    source = url,
    mount_point = "/mnt/logs",
    extra_configs = {config:sas}
  )

Out[20]: True

/tmp/custom_log.log

In [0]:
  storage_account_access_key=<storage_account_access_key>
  spark.conf.set(
    "fs.azure.account.key."+storage_account_name+".blob.core.windows.net",
    storage_account_access_key)

In [0]:
%fs
ls FileStore/tables/

path,name,size
dbfs:/FileStore/tables/CpuLogData2019_09_16-1.csv,CpuLogData2019_09_16-1.csv,177697
dbfs:/FileStore/tables/CpuLogData2019_09_16-2.csv,CpuLogData2019_09_16-2.csv,177697
dbfs:/FileStore/tables/CpuLogData2019_09_16-3.csv,CpuLogData2019_09_16-3.csv,177697
dbfs:/FileStore/tables/CpuLogData2019_09_16.csv,CpuLogData2019_09_16.csv,177697
dbfs:/FileStore/tables/CpuLogData2019_09_17-1.csv,CpuLogData2019_09_17-1.csv,299062
dbfs:/FileStore/tables/CpuLogData2019_09_17-2.csv,CpuLogData2019_09_17-2.csv,299062
dbfs:/FileStore/tables/CpuLogData2019_09_17-3.csv,CpuLogData2019_09_17-3.csv,299062
dbfs:/FileStore/tables/CpuLogData2019_09_17.csv,CpuLogData2019_09_17.csv,299062
dbfs:/FileStore/tables/CpuLogData2019_09_18-1.csv,CpuLogData2019_09_18-1.csv,286264
dbfs:/FileStore/tables/CpuLogData2019_09_18-2.csv,CpuLogData2019_09_18-2.csv,286264


In [0]:
import pyspark.sql.functions as F
from pyspark.sql.functions import *

try:
  # File location and type
  file_location = ["/FileStore/tables/CpuLogData2019_09_21.csv",\
                   "/FileStore/tables/CpuLogData2019_09_20.csv",\
                   "/FileStore/tables/CpuLogData2019_09_19.csv",\
                   "/FileStore/tables/CpuLogData2019_09_18.csv",\
                   "/FileStore/tables/CpuLogData2019_09_17.csv",\
                   "/FileStore/tables/CpuLogData2019_09_16.csv"]

  file_type = "csv"

  # CSV options
  infer_schema = "true"
  first_row_is_header = "true"
  delimiter = ","

  # The applied options are for CSV files. For other file types, these will be ignored.
  df = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .option("header", first_row_is_header) \
    .option("sep", delimiter) \
    .load(file_location)

  df_CPU = df.select("DateTime","keyboard","mouse","user_name")\
  .withColumn('Date', split(df["DateTime"]," ").getItem(0))\
  .withColumn('Time', split(df["DateTime"]," ").getItem(1))\
  .withColumn("DateTime",to_timestamp(df["DateTime"]))
  display(df_CPU).head(10)

  #df_CPU_21=df.select("DateTime","user_name","keyboard","mouse")
  #rows = df.count()
  #rows
  logger.info("load Successful!")
  
except Exception as e:
  logger.error("load Unsuccessful!")


DateTime,keyboard,mouse,user_name,Date,Time
2019-09-19T08:40:02.000+0000,1.0,32.0,iamnzm@outlook.com,2019-09-19,08:40:02
2019-09-19T08:45:02.000+0000,0.0,0.0,iamnzm@outlook.com,2019-09-19,08:45:02
2019-09-19T08:50:01.000+0000,0.0,0.0,iamnzm@outlook.com,2019-09-19,08:50:01
2019-09-19T08:55:01.000+0000,11.0,900.0,iamnzm@outlook.com,2019-09-19,08:55:01
2019-09-19T09:00:01.000+0000,2.0,25.0,iamnzm@outlook.com,2019-09-19,09:00:01
2019-09-19T09:05:01.000+0000,37.0,336.0,iamnzm@outlook.com,2019-09-19,09:05:01
2019-09-19T09:05:01.000+0000,0.0,55.0,deepshukla292@gmail.com,2019-09-19,09:05:01
2019-09-19T09:10:01.000+0000,0.0,136.0,iamnzm@outlook.com,2019-09-19,09:10:01
2019-09-19T09:10:01.000+0000,6.0,1112.0,deepshukla292@gmail.com,2019-09-19,09:10:01
2019-09-19T09:15:02.000+0000,0.0,84.0,iamnzm@outlook.com,2019-09-19,09:15:02


2021-07-29 06:01:54,590 - custom_log - ERROR - load Unsuccessful!

In [0]:
permanent_table_name = "CpuLogDataUsers1"

try:
  df_CPU.write.format("parquet").saveAsTable(permanent_table_name)
  logger.info("Permanent table created!")

except Exception:
  logger.error("Table creation unsuccessful!")

2021-07-29 06:03:55,478 - custom_log - INFO - Permanent table created!

In [0]:
%sql
show databases

databaseName
default


In [0]:
%sql
use default

In [0]:
%sql
show tables

database,tableName,isTemporary
default,cpulogdata,false
default,cpulogdata1,false
default,cpulogdatausers,false
default,custom_logging,false
,session_start_time_table,true
,user_active_time_table,true


In [0]:
# Finding users with lowest number of average hours 

try:
  lowest_avg_working_hours = sqlContext.sql("SELECT user_name,from_unixtime(round(((((count(*)-1)*5)*60)/6),2),'HH:mm') active_hours FROM CpuLogDataUsers1 WHERE keyboard !=0 OR mouse !=0 GROUP BY user_name ORDER BY active_hours asc")
  display(lowest_avg_working_hours)
  
  logger.info("Users with lowest number of average hours generated!")
  
except Exception:
  logger.error("Execution Failed!")

user_name,active_hours
damodharn21@gmail.com,02:38
bhagyashrichalke21@gmail.com,05:00
markfernandes66@gmail.com,05:23
rahilstar11@gmail.com,05:31
salinabodale73@gmail.com,06:05
sharlawar77@gmail.com,06:20
iamnzm@outlook.com,06:21
deepshukla292@gmail.com,06:35


2021-07-29 06:04:27,813 - custom_log - INFO - Users with lowest number of average hours generated!

In [0]:
# Finding users with highest number of average hours 

try:
  highest_avg_working_hours = sqlContext.sql("SELECT user_name,from_unixtime(round(((((count(*)-1)*5)*60)/6),2),'HH:mm') active_hours FROM CpuLogDataUsers1 WHERE keyboard !=0 OR mouse !=0 GROUP BY user_name ORDER BY active_hours desc")
  display(highest_avg_working_hours)
  logger.info("Users with highest number of average hours generated!")
  
except Exception:
  logger.error("Execution Failed!")

user_name,active_hours
deepshukla292@gmail.com,06:35
iamnzm@outlook.com,06:21
sharlawar77@gmail.com,06:20
salinabodale73@gmail.com,06:05
rahilstar11@gmail.com,05:31
markfernandes66@gmail.com,05:23
bhagyashrichalke21@gmail.com,05:00
damodharn21@gmail.com,02:38


2021-07-29 06:04:50,757 - custom_log - INFO - Users with highest number of average hours generated!

In [0]:
#Finding users with highest number of idle hours

try:
  highest_idle_hours = sqlContext.sql("SELECT user_name, count(*) no_of_times,from_unixtime(round(((((count(*)-1)*5)*60)/6),2),'HH:mm') idle_hours FROM CpuLogDataUsers1 WHERE keyboard==0 AND mouse==0 GROUP BY user_name ORDER BY idle_hours desc")
  display(highest_idle_hours)
  logger.info("Users with highest number of idle hours generated!")
  
except Exception:
  logger.error("Execution Failed!")

user_name,no_of_times,idle_hours
iamnzm@outlook.com,155,02:08
rahilstar11@gmail.com,152,02:05
salinabodale73@gmail.com,129,01:46
sharlawar77@gmail.com,123,01:41
bhagyashrichalke21@gmail.com,121,01:40
markfernandes66@gmail.com,119,01:38
deepshukla292@gmail.com,90,01:14
damodharn21@gmail.com,62,00:50


2021-07-29 06:04:59,218 - custom_log - INFO - Users with highest number of idle hours generated!

In [0]:
#Login Time of Each Day
try:
  session_start_time = sqlContext.sql("SELECT Date,to_timestamp(Min(DateTime)) min_time FROM CpuLogDataUsers1 GROUP BY Date ORDER BY Date")
  temp_table = "session_start_time_table"
  session_start_time.createOrReplaceTempView(temp_table)
  display(session_start_time)
  logger.info("Table created!")
  
except Exception:
  logger.error("Table creation failed!")

Date,min_time
2019-09-16,2019-09-16T12:55:01.000+0000
2019-09-17,2019-09-17T08:25:01.000+0000
2019-09-18,2019-09-18T08:30:01.000+0000
2019-09-19,2019-09-19T08:40:02.000+0000
2019-09-20,2019-09-20T09:05:01.000+0000
2019-09-21,2019-09-21T09:10:01.000+0000


2021-07-29 06:05:12,330 - custom_log - INFO - Table created!

In [0]:
#User active time
try:
  user_active_time = sqlContext.sql("SELECT user_name,Date,to_timestamp(Min(DateTime)) active_time From CpuLogDataUsers1 WHERE keyboard !=0 OR mouse !=0 GROUP BY user_name,Date ORDER BY Date")
  temp_table_name = "user_active_time_table"
  user_active_time.createOrReplaceTempView(temp_table_name)
  display(user_active_time)
  logger.info("Table Created!")
  
except Exception:
  logger.error("Table Creation Failed!")

user_name,Date,active_time
deepshukla292@gmail.com,2019-09-16,2019-09-16T13:00:01.000+0000
salinabodale73@gmail.com,2019-09-16,2019-09-16T12:55:02.000+0000
rahilstar11@gmail.com,2019-09-16,2019-09-16T13:00:03.000+0000
iamnzm@outlook.com,2019-09-16,2019-09-16T13:00:01.000+0000
bhagyashrichalke21@gmail.com,2019-09-16,2019-09-16T12:55:01.000+0000
sharlawar77@gmail.com,2019-09-16,2019-09-16T13:00:04.000+0000
markfernandes66@gmail.com,2019-09-17,2019-09-17T10:50:01.000+0000
deepshukla292@gmail.com,2019-09-17,2019-09-17T09:30:01.000+0000
sharlawar77@gmail.com,2019-09-17,2019-09-17T10:45:02.000+0000
salinabodale73@gmail.com,2019-09-17,2019-09-17T10:15:01.000+0000


2021-07-29 06:05:28,026 - custom_log - INFO - Table Created!

In [0]:
try:
  start_login = sqlContext.sql("SELECT user_active_time_table.active_time,session_start_time_table.min_time FROM user_active_time_table,session_start_time_table WHERE user_active_time_table.Date=session_start_time_table.Date")
  display(start_login)
  logger.info("Execution Successful!")
  
except Exception:
  logger.error("Execution Failed!")

active_time,min_time
2019-09-19T09:10:01.000+0000,2019-09-19T08:40:02.000+0000
2019-09-19T10:10:01.000+0000,2019-09-19T08:40:02.000+0000
2019-09-19T10:20:01.000+0000,2019-09-19T08:40:02.000+0000
2019-09-19T10:20:01.000+0000,2019-09-19T08:40:02.000+0000
2019-09-19T10:30:08.000+0000,2019-09-19T08:40:02.000+0000
2019-09-19T10:35:03.000+0000,2019-09-19T08:40:02.000+0000
2019-09-19T08:40:02.000+0000,2019-09-19T08:40:02.000+0000
2019-09-19T09:05:01.000+0000,2019-09-19T08:40:02.000+0000
2019-09-17T09:30:01.000+0000,2019-09-17T08:25:01.000+0000
2019-09-17T10:15:01.000+0000,2019-09-17T08:25:01.000+0000


2021-07-29 06:05:38,579 - custom_log - INFO - Execution Successful!

In [0]:
# Finding users with highest number of late comers
try:
  highest_late_comings = sqlContext.sql("SELECT td.user_name,from_unixtime(Round((Sum(td.diff_time))/6,2),'HH:mm')AS avg_time,count(*) As no_of_times_late FROM(SELECT user_active_time_table.user_name,(unix_timestamp(user_active_time_table.active_time)- unix_timestamp(session_start_time_table.min_time))diff_time FROM user_active_time_table,session_start_time_table WHERE user_active_time_table.Date=session_start_time_table.Date) AS td WHERE td.diff_time !=0 GROUP BY td.user_name ORDER BY avg_time desc")

  display(highest_late_comings)
  logger.info("Users with highest number of late comers tag generated!")
  
except Exception:
  logger.error("Execution Failed!")

user_name,avg_time,no_of_times_late
salinabodale73@gmail.com,01:26,6
bhagyashrichalke21@gmail.com,01:26,5
rahilstar11@gmail.com,01:23,6
markfernandes66@gmail.com,01:09,5
sharlawar77@gmail.com,01:07,5
damodharn21@gmail.com,00:59,3
iamnzm@outlook.com,00:34,4
deepshukla292@gmail.com,00:21,4


2021-07-29 06:05:46,122 - custom_log - INFO - Users with highest number of late comers tag generated!

In [0]:
dbutils.fs.mv("file:" + p_logfile, "dbfs:/FileStore/CustomLogging/" + p_filename)

Out[15]: True

In [0]:
dbutils.fs.ls("/FileStore/CustomLogging")

Out[16]: [FileInfo(path='dbfs:/FileStore/CustomLogging/custom_log.log', name='custom_log.log', size=965)]

In [0]:
%sql
DROP TABLE IF EXISTS CUSTOM_LOGGING;

CREATE TABLE CUSTOM_LOGGING
USING TEXT OPTIONS(path '/FileStore/CustomLogging/*', header = True)

In [0]:
%sql
SELECT * FROM CUSTOM_LOGGING;

value
"2021-07-29 06:01:54,590 - custom_log - ERROR - load Unsuccessful!"
"2021-07-29 06:03:04,353 - custom_log - ERROR - Table creation unsuccessful!"
"2021-07-29 06:03:55,478 - custom_log - INFO - Permanent table created!"
"2021-07-29 06:04:24,676 - custom_log - INFO - Users with lowest number of average hours generated!"
"2021-07-29 06:04:27,813 - custom_log - INFO - Users with lowest number of average hours generated!"
"2021-07-29 06:04:40,356 - custom_log - ERROR - Execution Failed!"
"2021-07-29 06:04:50,757 - custom_log - INFO - Users with highest number of average hours generated!"
"2021-07-29 06:04:59,218 - custom_log - INFO - Users with highest number of idle hours generated!"
"2021-07-29 06:05:12,330 - custom_log - INFO - Table created!"
"2021-07-29 06:05:28,026 - custom_log - INFO - Table Created!"
